In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

from tensorflow.contrib import learn
from sklearn.metrics import mean_squared_error

#from lstm import lstm_model
from data_processing import generate_data

In [2]:
def lstm_model(features, labels, mode, params):
    # connect first hidden layer to input layer
    # input features is (N, STEPS, 1) shape
    # label y is (N, 1) shape
    # here N is points number, STEPS==3, as for 28x28 image, step is 28
    # so first_hidden_layer is (N, steps, 10) shape
    #first_hidden_layer = tf.layers.dense(X, 10, activation=tf.nn.relu)

    # lstm cells
    lstm_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=5)]
    stacked_lstm = tf.contrib.rnn.MultiRNNCell(lstm_cells, state_is_tuple=True)

    # reshape X, why?
    # x_ is [(N,1), (N,1), (N,1)]
    x_ = tf.unstack(features["X"], axis=1)
    # For static_rnn, inputs x_ is a length T list of Tensor shape [batch_size, input_size], so here T==3, batch_size == N, input_size==1??
    # output is a length T of outputs (one for each input), or a nested tuple of such
    # so output is [(N,1), (N,1), (N,1)]
    output, state = tf.contrib.rnn.static_rnn(stacked_lstm, x_, dtype=np.float32)
    # dnn layers, not used 
    #output_layer = tf.layers.dense(output, 1)
    #predictions = tf.reshape(output_layer, [-1])
    #loss = tf.losses.mean_squared_error(labels, predictions)
    #output = tf.reshape(output, [-1])
    output = output[-1]
    predictions, loss = tf.contrib.learn.models.linear_regression(output, labels)

    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        loss = loss
    else:
        loss = None

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    else:
        train_op = None

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = predictions
    else:
        predictions = None

    return tf.estimator.EstimatorSpec(
             mode=mode,
             predictions=predictions,
             loss=loss, 
             train_op = train_op)

In [4]:
STEPS = 3
TRAINING_STEPS = 10000
PRINT_STEPS = TRAINING_STEPS / 10
BATCH_SIZE = 100
LEARNING_RATE = 0.01

X, y = generate_data(np.sin, np.linspace(0, 100, 10000, dtype=np.float32), STEPS, seperate=False)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"X" : X['train']},
        y=y['train'],
        shuffle=True)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"X" : X['test']},
        y=y['test'],
        shuffle=False)

In [5]:
model_params = {"learning_rate": LEARNING_RATE}
nn = tf.estimator.Estimator(model_fn=lstm_model, params=model_params)
nn.train(input_fn=train_input_fn, steps=100)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpy0yts2ey', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
Instructions for updating:
Use `tf.contrib.losses.mean_squared_error` and explicit logits computation.
Instructions for updating:
Use tf.losses.mean_squared_error instead.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpy0yts2ey/model.ckpt.
INFO:tensorflow:loss = 1.18838, step = 1
INFO:tensorflow:Saving checkpoints for 64 into /tmp/tmpy0yts2ey/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00531394.


In [8]:
predicted = nn.predict(input_fn=test_input_fn)

print(dir(predicted))
for i,p in enumerate(predicted):
    print("Prediction %s : %s"%(i+1, p))

['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


ValueError: Tried to convert 'values' to a tensor and failed. Error: None values not supported.

In [ ]:

rmse = np.sqrt(((predicted - y['test']) ** 2).mean(axis=0))
score = mean_squared_error(predicted, y['test'])
print ("MSE: %f" % score)

plot_predicted, = plt.plot(predicted, label='predicted')
plot_test, = plt.plot(y['test'], label='test')
plt.legend(handles=[plot_predicted, plot_test])
plt.show()